In [58]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
!pip install folium
import folium
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

Getting the url

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result=requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Mon, 07 Dec 2020 19:32:12 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.8', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'X-Request-Id': 'cf0149f1-3bc5-409e-b9e6-d59a5559a1a8', 'Last-Modified': 'Sun, 06 Dec 2020 20:02:49 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '47608', 'X-Cache': 'cp1081 miss, cp1077 hit/25', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Report-To': '{ "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'NEL': '{ "report_to": "wm_nel", "max_age": 86400, "failu

Getting data from URL and cleaning it

In [3]:
soup=BeautifulSoup(result.content,'html.parser')
table=soup.find('table')
trs=table.find_all('tr')
rows=[]
for tr in trs:
    i=tr.find_all('td')
    if(i):
        rows.append(i)
lst=[]
for row in rows:
    postalcode=row[0].text.rstrip()
    borough=row[1].text.rstrip()
    neighbourhood=row[2].text.rstrip()
    if(borough!='Not assigned'):
        if(neighbourhood=='Not Assigned'):
            neighbourhood=borough
        lst.append([postalcode,borough,neighbourhood])
lst

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Convert the above array into a dataframe

In [4]:
cols=['PostalCode','Borough','Neighbourhood']
df=pd.DataFrame(lst,columns=cols)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
print(df.shape)

(103, 3)


In [6]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
dfgeo= pd.read_csv(r'http://cocl.us/Geospatial_data')
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
dfgeo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two dataframes df and dfgeo

In [8]:
df2=pd.merge(df,dfgeo,on="PostalCode",how="left")

In [9]:
df2.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
geolocator=Nominatim(user_agent="coursera")
address='Toronto'
try:
    location=geolocator.geocode(address)
    latitude=location.latitude
    longitude=location.longitude
    print('The geographical coordinates of {} are latitude:{},longitude:{}'.format(address,latitude,longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map=folium.Map(location=[latitude,longitude],zoom_start=11)


for lat, lng, label in zip(df2['Latitude'],df2['Longitude'],df2['PostalCode']):
    label=folium.Popup(label)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill_color='red',
    fill_opacity=0.7
    ).add_to(my_map)
    

my_map

The geographical coordinates of Toronto are latitude:43.6534817,longitude:-79.3839347


In [10]:
CLIENT_ID="SD4JVBJ5GULNKTCOUD5ZBFHSX3QMOBEBB01B4TSASDA3LHDL"
CLIENT_SECRET="0YNY5TTT3OLZSU4VAGHU43VEUNVNACZQVINI52IFARARUFVX"
VERSION = '20180605'

In [11]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [12]:
LIMIT = 100 
radius = 500 
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=SD4JVBJ5GULNKTCOUD5ZBFHSX3QMOBEBB01B4TSASDA3LHDL&client_secret=0YNY5TTT3OLZSU4VAGHU43VEUNVNACZQVINI52IFARARUFVX&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [13]:
results = requests.get(url).json()

In [14]:
results

{'meta': {'code': 200, 'requestId': '5fcf3d45e915b64c9362a54f'},
 'response': {'headerLocation': 'Rexdale',
  'headerFullLocation': 'Rexdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.7439164045, 'lng': -79.58222007762089},
   'sw': {'lat': 43.734916395499994, 'lng': -79.59465372237912}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4be58dc4cf200f479154133c',
       'name': 'Shoppers Drug Mart',
       'location': {'address': '1530 Albion Rd',
        'crossStreet': 'Albion Mall',
        'lat': 43.741685,
        'lng': -79.584487,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.741685,
          'lng': -79.584487}],
        'distance': 405,
        'postalCode': 'M9V 1B4'

In [15]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4be58dc4cf200f479154133c',
   'name': 'Shoppers Drug Mart',
   'location': {'address': '1530 Albion Rd',
    'crossStreet': 'Albion Mall',
    'lat': 43.741685,
    'lng': -79.584487,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.741685,
      'lng': -79.584487}],
    'distance': 405,
    'postalCode': 'M9V 1B4',
    'cc': 'CA',
    'city': 'Etobicoke',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['1530 Albion Rd (Albion Mall)',
     'Etobicoke ON M9V 1B4',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d10f951735',
     'name': 'Pharmacy',
     'pluralName': 'Pharmacies',
     'shortName': 'Pharmacy',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pharmacy_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  

In [41]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4be58dc4cf200f479154133c-0,0,"[{'summary': 'This spot is popular', 'type': '...",4be58dc4cf200f479154133c,Shoppers Drug Mart,1530 Albion Rd,Albion Mall,43.741685,-79.584487,"[{'label': 'display', 'lat': 43.741685, 'lng':...",405,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[]
1,e-0-4c633939e1621b8d48842553-1,0,"[{'summary': 'This spot is popular', 'type': '...",4c633939e1621b8d48842553,Subway,"6210 Finch Ave West, Store 103",at Albion Rd.,43.742645,-79.589643,"[{'label': 'display', 'lat': 43.74264512142215...",372,M9V 0A1,CA,Toronto,ON,Canada,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[]
2,e-0-4be70e26cf200f47e334153c-2,0,"[{'summary': 'This spot is popular', 'type': '...",4be70e26cf200f47e334153c,Popeyes Louisiana Kitchen,1530 Albion Rd.,at Kipling Ave. (Albion Centre),43.741202,-79.584545,"[{'label': 'display', 'lat': 43.74120165509377...",370,M9V 1B4,CA,Etobicoke,ON,Canada,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[]
3,e-0-4cd4738cdfb4a1cd4337535c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4cd4738cdfb4a1cd4337535c,The Beer Store,1530 Albion Rd,Near Finch Ave. W.,43.741694,-79.584373,"[{'label': 'display', 'lat': 43.7416936, 'lng'...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[]
4,e-0-4b04a05bf964a520c45522e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b04a05bf964a520c45522e3,Sheriff's No Frills,1530 Albion Rd,at Finch Ave. W.,43.741696,-79.584379,"[{'label': 'display', 'lat': 43.741696, 'lng':...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
5,e-0-5112b872e4b0c0a78d7bcd27-5,0,"[{'summary': 'This spot is popular', 'type': '...",5112b872e4b0c0a78d7bcd27,Sunny Foodmart,1620 Albion road,Albion Road and Finch Ave,43.741840,-79.590561,"[{'label': 'display', 'lat': 43.74184023292111...",319,NaN,CA,Toronto,ON,Canada,"[1620 Albion road (Albion Road and Finch Ave),...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
6,e-0-4d8ba6910c4e41bdaaf7667f-6,0,"[{'summary': 'This spot is popular', 'type': '...",4d8ba6910c4e41bdaaf7667f,Pizza Pizza,"1530 Albion Road, Unit T25",NaN,43.741569,-79.584489,"[{'label': 'display', 'lat': 43.74156896801906...",397,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
7,e-0-4c1951d6834e2d7f2d3a2a80-7,0,"[{'summary': 'This spot is popular', 'type': '...",4c1951d6834e2d7f2d3a2a80,McDonald's,"1530 Albion Road, Unit F-1",NaN,43.741757,-79.584230,"[{'label': 'display', 'lat': 43.7417571, 'lng'...",427,M9V 1B4,CA,Toronto,ON,Canada,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]
8,e-0-4da73b855da37feee89a1143-8,0,"[{'summary': 'This spot is popular', 'type': '...",4da73b855da37feee89a1143,Pizza Nova,1701 Martingrove Road,NaN,43.736761,-79.589817,"[{'label': 'display', 'lat': 43.73676056243852...",315,M9V 4N4,CA,Toronto,ON,Canada,"[1701 Martingrove Road, Toronto ON M9V 4N4, Ca...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
9,e-0-4ce971bff8653704e40db7c4-9,0,"[{'summary': 'This spot is popular', 'type': '...",4ce971bff8653704e40db7c4,LCBO,Albion Mall,1530 Albion Rd,43.741508,-79.584501,"[{'label': 'display', 'lat': 4

In [42]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
1,Subway,Sandwich Place,43.742645,-79.589643
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,The Beer Store,Beer Store,43.741694,-79.584373
4,Sheriff's No Frills,Grocery Store,43.741696,-79.584379
5,Sunny Foodmart,Grocery Store,43.741840,-79.590561
6,Pizza Pizza,Pizza Place,43.741569,-79.584489
7,McDonald's,Fast Food Restaurant,43.741757,-79.584230
8,Pizza Nova,Pizza Place,43.736761,-79.589817
9,LCBO,Liquor Store,43.741508,-79.584501


In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## 

In [44]:
df2

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


In [45]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

In [46]:
print(venues.shape)
venues.head()

(2141, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [48]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,104,104,104,104,104,104
Downtown Toronto,1248,1248,1248,1248,1248,1248
East Toronto,119,119,119,119,119,119
East York,79,79,79,79,79,79
Etobicoke,74,74,74,74,74,74
Mississauga,13,13,13,13,13,13
North York,241,241,241,241,241,241
Scarborough,90,90,90,90,90,90
West Toronto,153,153,153,153,153,153


In [49]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 273 unique categories.


In [50]:

# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
onehot.shape

(2141, 273)

In [52]:

grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009615,...,0.000000,0.00,0.000000,0.000000,0.009615,0.000000,0.000000,0.000000,0.000000,0.009615
1,Downtown Toronto,0.000000,0.000801,0.000801,0.000801,0.000801,0.001603,0.001603,0.000801,0.013622,...,0.002404,0.00,0.011218,0.001603,0.004006,0.000000,0.006410,0.000000,0.000000,0.005609
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025210,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.008403,0.000000,0.000000,0.016807
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.012658
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004149,0.000000,0.004149,0.000000,0.000000,0.000000,0.000000,0.000000,0.008299,...,0.000000,0.00,0.000000,0.004149,0.008299,0.000000,0.000000,0.000000,0.016598,0.000000
7,Scarborough,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,...,0.011111,0.00,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.013072,0.000000,0.013072,0.000000,0.006536,0.000000,0.000000,0.013072
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000


In [53]:
grouped.shape

(10, 273)

In [54]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2            Park  0.06
3            Café  0.06
4     Pizza Place  0.05


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.05
2          Restaurant  0.04
3               Hotel  0.03
4  Seafood Restaurant  0.02


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1    Greek Restaurant  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4              Bakery  0.03


----East York----
            venue  freq
0    Intersection  0.05
1     Coffee Shop  0.05
2            Bank  0.05
3  Sandwich Place  0.04
4            Park  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1     Coffee Shop  0.07
2  Sandwich Place  0.07
3        Pharmacy  0.05
4             Gym  0.04


----Mississauga----
                      venue  freq
0                     Hotel  0.15
1               Coffee Shop 

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Restaurant,Bagel Shop,Italian Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Brewery,Park,Ice Cream Shop,Restaurant,Bookstore,Café,Pub
3,East York,Intersection,Coffee Shop,Bank,Burger Joint,Sandwich Place,Sporting Goods Shop,Pizza Place,Park,Indian Restaurant,Pet Store
4,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Fast Food Restaurant,Grocery Store,Gym,Liquor Store,Café,Convenience Store
5,Mississauga,Coffee Shop,Hotel,Middle Eastern Restaurant,Gym,Intersection,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,American Restaurant,Gas Station
6,North York,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Grocery Store,Park,Bank,Juice Bar
7,Scarborough,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Indian Restaurant,Breakfast Spot,Playground
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Pizza Place,Gift Shop,Grocery Store
9,York,Park,Breakfast Spot,Hockey Arena,Bar,Dog Run,Bus Line,Pool,Tennis Court,Sandwich Place,Discount Store


In [59]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1], dtype=int32)